In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.offline as pyo
import plotly.graph_objs as go
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score


In [2]:
df_train = pd.read_csv('/kaggle/input/playground-series-s4e8/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e8/test.csv')
df_sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e8/sample_submission.csv')

In [3]:
for i in df_train.columns:
    if df_train[i].dtype == object:
        print(i)
        print(df_train[i].unique())

class
['e' 'p']
cap-shape
['f' 'x' 'p' 'b' 'o' 'c' 's' 'd' 'e' 'n' nan 'w' 'k' 'l' '19.29' '5 f' 't'
 'g' 'z' 'a' '2.85' '7 x' 'r' 'u' '3.55' 'is s' 'y' '4.22' '3.6' '21.56'
 'i' '6 x' '24.16' '8' 'm' 'ring-type' '10.13' 'is p' '7.43' 'h' '0.82'
 '10.46' '2.77' '2.94' '12.62' '5.15' '19.04' '4.97' '49.21' 'b f' '9.13'
 '1.66' '3.37' '7.21' '3.25' '11.12' '3 x' '4.3' '7.41' '6.21' '8.29'
 '54.78' '20.25' '3.52' '3.04' '2.63' '3.91' '6.44' '8.3' '7.6' '17.44'
 '4.33' '2.82' '6.53' '19.06']
cap-surface
['s' 'h' 'y' 'l' 't' 'e' 'g' nan 'd' 'i' 'w' 'k' '15.94' 'f' 'n' 'r' 'o'
 'a' 'u' 'z' '2.7' 'does l' '5.07' 'p' 'b' 'm' 'cap-diameter' '1.43' 'x'
 '7.14' 'c' 'is h' 'does t' '0.85' '6.57' '12.79' '6.45' '4.66' '23.18'
 '3.06' '16.39' '4.21' 'veil-color' '11.78' '8.1' 'has-ring' 'does h'
 '1.42' 'class' 'has h' 'does None' '10.83' 'season' '8.96' '14.04' '5.73'
 'is None' '24.38' '2.81' '0.88' '2.11' '2.79' 'ring-type'
 'does-bruise-or-bleed' '4.93' 'spore-print-color' 'spore-color' '2.92'
 

In [4]:
for i in df_test.columns:
    if df_test[i].dtype == object:
        print(i)
        print(df_test[i].unique())

cap-shape
['x' 'o' 'b' 'f' 's' 'p' 'c' 'g' 't' 'i' 'a' 'n' '7.21' 'e' nan 'w' 'd'
 'k' 'u' 'y' '3.34' '8.3' 'r' 'l' '3.32' '19.76' 'is f' '0.93' '5 f' 'z'
 'p f' 'h' '1.25' 'm' '4.55' 'ring-type' '11.43' '6' '5.59' '4.41' '3.73'
 '3.47' '6.75' '51.63' '6.52' '9.49' '2.7' '3.15' '5.05' 'cap-diameter'
 '4.51' '4.86' '0.97' '3.94' '6.47' '3.23' '2.02' '1.53' '8' '6.15'
 '17.89' '0.74' '4.54']
cap-surface
[nan 't' 'g' 'h' 'd' 'e' 'y' 'k' 's' 'i' 'l' 'w' 'r' 'f' 'o' 'm' 'a' 'n'
 'p' 'b' 'x' 'season' 'u' '6.04' 'z' '10.09' 'spore-print-color' 'c'
 '2.97' 'class' '7.78' '5.05' '5.92' '5.98' '11.96' '7.96' '9.53' '17.98'
 'habitat' '2.47' '20.64' '8.34' 'has-ring' 'does None' '1.95' '16.48'
 '5.84' '5.93' '12.3' '14.18' '4.87' '15.52' '3.49' '6.58' '12.15' '6.7'
 '4.91' '0.94' '2.84' '5.1']
cap-color
['n' 'o' 'y' 'k' 'b' 'w' 'e' 'g' 'p' 'u' 'l' 'r' 'z' 'f' nan 's' 'd' 'i'
 '10.93' 'ring-type' 'a' 'x' '0.73' 'c' 't' '19.18' 'h' '5.35' 'm' '21.38'
 'has-ring' '5.38' 'class' '7.41' '18.49' 'habit

In [5]:
i_list = {}
for i in df_train.columns:
    if df_train[i].dtype == object :
        i_list[i] = df_train[i].value_counts().to_dict()
        print(i_list[i])

{'p': 1705396, 'e': 1411549}
{'x': 1436026, 'f': 676238, 's': 365146, 'b': 318646, 'o': 108835, 'p': 106967, 'c': 104520, 'd': 65, 'e': 60, 'n': 41, 't': 36, 'w': 36, 'g': 34, 'y': 33, 'r': 32, 'a': 30, 'u': 27, 'z': 22, 'l': 21, 'i': 14, 'k': 13, 'h': 6, 'm': 2, '8': 2, '3.55': 2, '7 x': 2, '4.97': 2, '3.25': 1, '11.12': 1, '3 x': 1, '4.3': 1, '7.41': 1, '6.21': 1, '8.29': 1, '7.21': 1, '54.78': 1, '2.82': 1, '20.25': 1, '3.52': 1, '3.04': 1, '6.53': 1, '2.63': 1, '3.91': 1, '6.44': 1, '8.3': 1, '7.6': 1, '1.66': 1, '17.44': 1, '4.33': 1, '3.37': 1, '7.43': 1, '9.13': 1, 'ring-type': 1, '19.29': 1, '5 f': 1, '2.85': 1, 'is s': 1, '4.22': 1, '3.6': 1, '21.56': 1, '6 x': 1, '24.16': 1, '10.13': 1, 'b f': 1, 'is p': 1, '0.82': 1, '10.46': 1, '2.77': 1, '2.94': 1, '12.62': 1, '5.15': 1, '19.04': 1, '49.21': 1, '19.06': 1}
{'t': 460777, 's': 384970, 'y': 327826, 'h': 284460, 'g': 263729, 'd': 206832, 'k': 128875, 'e': 119712, 'i': 113440, 'w': 109840, 'l': 44859, 'n': 96, 'f': 75, 'p': 59,

In [6]:
test_list = {}
for i in df_test.columns:
    if df_test[i].dtype == object :
        test_list[i] = df_test[i].value_counts().to_dict()
        print(test_list[i])

{'x': 957949, 'f': 452360, 's': 242698, 'b': 211879, 'o': 71972, 'p': 71302, 'c': 69436, 'e': 33, 'd': 30, 't': 28, 'n': 27, 'w': 27, 'y': 24, 'a': 24, 'u': 18, 'g': 17, 'l': 17, 'k': 17, 'r': 16, 'z': 8, 'm': 4, 'i': 4, 'h': 3, 'is f': 2, '2.7': 1, '3.15': 1, '5.05': 1, 'cap-diameter': 1, '4.51': 1, '4.86': 1, '0.97': 1, '3.94': 1, '17.89': 1, '6.15': 1, '6.47': 1, '3.23': 1, '2.02': 1, '1.53': 1, '8': 1, '0.74': 1, '6.52': 1, '9.49': 1, '1.25': 1, '51.63': 1, 'p f': 1, '7.21': 1, '3.34': 1, '8.3': 1, '3.32': 1, '19.76': 1, '0.93': 1, '5 f': 1, '4.55': 1, '6.75': 1, 'ring-type': 1, '11.43': 1, '6': 1, '5.59': 1, '4.41': 1, '3.73': 1, '3.47': 1, '4.54': 1}
{'t': 306852, 's': 257190, 'y': 218336, 'h': 189737, 'g': 176140, 'd': 137675, 'k': 86041, 'e': 79918, 'i': 75570, 'w': 73109, 'l': 30101, 'f': 54, 'n': 50, 'p': 44, 'o': 37, 'a': 28, 'x': 27, 'm': 25, 'r': 25, 'u': 21, 'c': 19, 'b': 14, 'z': 6, 'spore-print-color': 3, 'season': 3, 'does None': 2, '0.94': 1, '2.84': 1, '4.91': 1, '6.

In [7]:
filtered_train_list = {col: {value: count for value, count in counts.items() if count >= 10} for col, counts in i_list.items()}
filtered_test_list = {col: {value: count for value, count in counts.items() if count >= 10} for col, counts in test_list.items()}


In [8]:
for i in filtered_train_list:
    df_train[i] = df_train[i].apply(lambda x: x if x in filtered_train_list[i].keys() else np.nan)
for i in filtered_test_list:
    df_test[i] = df_test[i].apply(lambda x: x if x in filtered_test_list[i].keys() else np.nan)

In [9]:
df_train

,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,0,e,8.80,f,s,u,f,a,c,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,1,p,4.51,x,h,o,f,a,c,n,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,2,e,6.94,f,s,b,f,x,c,w,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,3,e,3.88,f,y,g,f,s,NaN,g,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,4,e,5.85,x,l,w,f,d,NaN,w,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3116940,3116940,e,9.29,f,NaN,n,t,NaN,NaN,w,...,b,NaN,w,u,w,t,g,NaN,d,u
3116941,3116941,e,10.88,s,NaN,w,t,d,c,p,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
3116942,3116942,p,7.82,x,e,e,f,a,NaN,w,...,NaN,NaN,y,NaN,w,t,z,NaN,d,a
3116943,3116943,e,9.45,p,i,n,t,e,NaN,p,...,NaN,y,w,NaN,NaN,t,p,NaN,d,u


In [10]:
for i in df_train.columns:
    print(i)
    print(df_train[i].unique())

id
[      0       1       2 ... 3116942 3116943 3116944]
class
['e' 'p']
cap-diameter
[ 8.8   4.51  6.94 ... 38.11 55.63 54.07]
cap-shape
['f' 'x' 'p' 'b' 'o' 'c' 's' 'd' 'e' 'n' nan 'w' 'k' 'l' 't' 'g' 'z' 'a'
 'r' 'u' 'y' 'i']
cap-surface
['s' 'h' 'y' 'l' 't' 'e' 'g' nan 'd' 'i' 'w' 'k' 'f' 'n' 'r' 'o' 'a' 'u'
 'p' 'b' 'm' 'x' 'c']
cap-color
['u' 'o' 'b' 'g' 'w' 'n' 'e' 'y' 'r' 'p' 'k' 'l' 'i' 'h' 'd' 's' 'a' 'f'
 nan 'c' 'x' 'm' 'z' 't']
does-bruise-or-bleed
['f' 't' nan 'w' 'c']
gill-attachment
['a' 'x' 's' 'd' 'e' nan 'f' 'p' 'l' 'm' 'b' 'n' 'g' 'i' 'u' 't' 'o' 'c'
 'w' 'k' 'r' 'h' 'y']
gill-spacing
['c' nan 'd' 'f' 'b' 'a' 'e' 's']
gill-color
['w' 'n' 'g' 'k' 'y' 'f' 'p' 'o' 'b' 'u' 'e' 'r' 'd' 't' nan 'z' 'h' 'x'
 's' 'c' 'm' 'l' 'a']
stem-height
[ 4.51  4.79  6.85 ... 26.09 47.33 26.53]
stem-width
[15.39  6.48  9.93 ... 66.91 79.92 53.44]
stem-root
[nan 'b' 'c' 'r' 's' 'f' 'y' 'k' 'd' 'n' 'w' 'p' 't' 'l' 'g']
stem-surface
[nan 'y' 's' 't' 'g' 'h' 'k' 'i' 'f' 'l' 'd' 'x' 'w' 'a'

In [11]:
for i in df_test.columns:
    print(i)
    print(df_test[i].unique())

id
[3116945 3116946 3116947 ... 5194906 5194907 5194908]
cap-diameter
[ 8.64  6.9   2.   ... 58.98 39.12 46.19]
cap-shape
['x' 'o' 'b' 'f' 's' 'p' 'c' 'g' 't' nan 'a' 'n' 'e' 'w' 'd' 'k' 'u' 'y'
 'r' 'l']
cap-surface
[nan 't' 'g' 'h' 'd' 'e' 'y' 'k' 's' 'i' 'l' 'w' 'r' 'f' 'o' 'm' 'a' 'n'
 'p' 'b' 'x' 'u' 'c']
cap-color
['n' 'o' 'y' 'k' 'b' 'w' 'e' 'g' 'p' 'u' 'l' 'r' nan 'f' 's' 'd' 'i' 'a'
 'x' 'c' 't' 'h']
does-bruise-or-bleed
['t' 'f' nan 'x']
gill-attachment
[nan 's' 'p' 'x' 'f' 'a' 'd' 'e' 'c' 't' 'i' 'u' 'k' 'n' 'w' 'l' 'b' 'o'
 'g' 'y']
gill-spacing
[nan 'c' 'f' 'd' 's' 'e' 'a']
gill-color
['w' 'y' 'n' 'e' 'f' 'b' 'g' 'p' 'o' 'u' 'k' 'r' 'l' 'd' nan 't' 's' 'a'
 'x' 'c' 'h']
stem-height
[11.13  1.27  6.18 ... 38.4   1.01 25.32]
stem-width
[17.12 10.75  3.14 ... 55.11 75.42 42.68]
stem-root
['b' nan 's' 'r' 'c' 'f' 'p' 'g' 'y']
stem-surface
[nan 's' 'g' 'y' 't' 'i' 'k' 'h' 'f' 'w' 'r' 'd' 'e' 'c' 'u' 'o' 'n' 'x']
stem-color
['w' 'n' 'y' 'g' 'o' 'p' 'u' 'e' 'k' 'r' 'l' 'b' 's' 'f

In [12]:
df_train.dtypes

id                        int64
class                    object
cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-root                object
stem-surface             object
stem-color               object
veil-type                object
veil-color               object
has-ring                 object
ring-type                object
spore-print-color        object
habitat                  object
season                   object
dtype: object

In [13]:
df_train['cap-diameter'].fillna(0, inplace = True)


/tmp/ipykernel_18/2765844019.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [14]:
df_train['cap-diameter'].isna().sum()

0

In [15]:
df_test['cap-diameter'].fillna(0, inplace = True)


/tmp/ipykernel_18/3837369161.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [16]:
df_train[['stem-height','stem-width']].isna().sum()

stem-height    0
stem-width     0
dtype: int64

In [17]:
df_test[['stem-height','stem-width']].isna().sum()

stem-height    1
stem-width     0
dtype: int64

In [18]:
df_test['stem-height'].fillna(0, inplace = True)


/tmp/ipykernel_18/3177765968.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [19]:
df_test.fillna('Missing',inplace = True)
df_train.fillna('Missing',inplace = True)


In [20]:
Y_train = df_train['class'].apply(lambda x: 1 if x=='p' else 0)


In [21]:
X_train = df_train.iloc[:,2:]

In [22]:
X_test = df_test.iloc[:,1:]

In [23]:
X_test

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,8.64,x,Missing,n,t,Missing,Missing,w,11.13,17.12,b,Missing,w,u,w,t,g,Missing,d,a
1,6.90,o,t,o,f,Missing,c,y,1.27,10.75,Missing,Missing,n,Missing,Missing,f,f,Missing,d,a
2,2.00,b,g,n,f,Missing,c,n,6.18,3.14,Missing,Missing,n,Missing,Missing,f,f,Missing,d,s
3,3.47,x,t,n,f,s,c,n,4.98,8.51,Missing,Missing,w,Missing,n,t,z,Missing,d,u
4,6.17,x,h,y,f,p,Missing,y,6.73,13.70,Missing,Missing,y,Missing,y,t,Missing,Missing,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,0.88,x,g,w,f,a,d,w,2.67,1.35,Missing,Missing,e,Missing,Missing,f,f,Missing,d,u
2077960,3.12,x,s,w,f,d,c,w,2.69,7.38,Missing,Missing,w,Missing,Missing,f,f,Missing,g,a
2077961,5.73,x,e,e,f,a,Missing,w,6.16,9.74,Missing,Missing,y,Missing,w,t,z,Missing,d,a
2077962,5.03,b,g,n,f,a,d,g,6.00,3.46,Missing,s,g,Missing,Missing,f,f,Missing,d,a


In [24]:
df_test

,id,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,3116945,8.64,x,Missing,n,t,Missing,Missing,w,11.13,...,b,Missing,w,u,w,t,g,Missing,d,a
1,3116946,6.90,o,t,o,f,Missing,c,y,1.27,...,Missing,Missing,n,Missing,Missing,f,f,Missing,d,a
2,3116947,2.00,b,g,n,f,Missing,c,n,6.18,...,Missing,Missing,n,Missing,Missing,f,f,Missing,d,s
3,3116948,3.47,x,t,n,f,s,c,n,4.98,...,Missing,Missing,w,Missing,n,t,z,Missing,d,u
4,3116949,6.17,x,h,y,f,p,Missing,y,6.73,...,Missing,Missing,y,Missing,y,t,Missing,Missing,d,u
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077959,5194904,0.88,x,g,w,f,a,d,w,2.67,...,Missing,Missing,e,Missing,Missing,f,f,Missing,d,u
2077960,5194905,3.12,x,s,w,f,d,c,w,2.69,...,Missing,Missing,w,Missing,Missing,f,f,Missing,g,a
2077961,5194906,5.73,x,e,e,f,a,Missing,w,6.16,...,Missing,Missing,y,Missing,w,t,z,Missing,d,a
2077962,5194907,5.03,b,g,n,f,a,d,g,6.00,...,Missing,s,g,Missing,Missing,f,f,Missing,d,a


In [25]:
enc_auto = TargetEncoder()

In [26]:
encode = enc_auto.fit(X_train, Y_train)

In [27]:
X_train_trans = enc_auto.transform(X_train)
X_test_trans = enc_auto.transform(X_test)

In [28]:
train_data = lgb.Dataset(X_train_trans, label=Y_train)


In [29]:
test_data = lgb.Dataset(X_test_trans)

In [30]:
params = {
    'objective': 'binary', 
    'metric': 'binary_error',  
    'boosting_type': 'gbdt',  
    'num_leaves': 1000,  
    'learning_rate': 0.01,  
    'feature_fraction': 0.95,  
    'verbose': -1  ,
    'max_bin ':1024
}


In [31]:
lgb_model = lgb.train(
    params,
    train_data,
    num_boost_round=1500,
    valid_sets=[train_data, test_data],
)

In [32]:
y_pred_prob = lgb_model.predict(X_test_trans, num_iteration=lgb_model.best_iteration)

In [33]:
y_pred = ['p' if x > 0.7 else 'e' for x in y_pred_prob]

In [34]:
output = pd.DataFrame({'id': df_sample_submission.id, 'class': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [35]:
output

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e
...,...,...
2077959,5194904,p
2077960,5194905,p
2077961,5194906,p
2077962,5194907,e
